In [ ]:
#Import packages
from polygon.rest import RESTClient
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd

#Import polygon API key from file
polygon_api_key = open("polygon_io_api_key.txt").read()

#Set Dates (free API only goes back 2 years)
current_date = dt.datetime.now().date()
past_date = current_date - relativedelta(years=2)

In [ ]:
#Run API
ticker = "aapl".upper()

client = RESTClient(polygon_api_key)

aggs = []
for a in client.list_aggs(
    ticker,
    1,
    "day",
    past_date,
    current_date,
    adjusted="true",
    sort="asc",
):
    aggs.append(a)

In [51]:
#Convert results to a dataframe
df = pd.DataFrame(aggs)

#transform timestamp to date
df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
df["date"] = df["date"].dt.date

#set index of df to date
df.set_index("date", inplace=True)
df["ticker"] = ticker
print(df)

               open      high      low   close       volume      vwap  \
date                                                                    
2023-08-14  177.970  179.6900  177.305  179.46   43675627.0  178.9475   
2023-08-15  178.880  179.4800  177.050  177.45   43622593.0  177.9890   
2023-08-16  177.130  178.5400  176.500  176.57   46964857.0  177.4365   
2023-08-17  177.140  177.5054  173.480  174.00   66062882.0  174.5857   
2023-08-18  172.300  175.1000  171.960  174.49   61172150.0  173.8845   
...             ...       ...      ...     ...          ...       ...   
2025-08-05  203.400  205.3400  202.160  202.92   44155079.0  203.5480   
2025-08-06  205.630  215.3800  205.590  213.25  108483103.0  212.8573   
2025-08-07  218.875  220.8500  216.580  220.03   90224834.0  219.3267   
2025-08-08  220.830  231.0000  219.250  229.35  113853967.0  227.3071   
2025-08-11  227.920  229.5600  224.760  227.18   61806132.0  227.3230   

                timestamp  transactions   otc tick